<a href="https://colab.research.google.com/github/Rohsha/CE888/blob/master/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer, confusion_matrix
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder

In [0]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv', header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-7.86,99.00,99.00,

In [0]:
## Copy values and keep
test_data = df.copy()
df = df.loc[:, 1:]  ##first column is just number of jokes, and not ratings
arr = df.values
print(arr.shape)

(24983, 100)


In [0]:
## Take all rated jokes. If not rated don't add.
## np.where returns an ndarray with all elements which satisfy the condition (x), and then all elements which don't satisfy the condition
## Over here first array is all rows, and 2nd array is all columns indices where it is not 99
## Use this to understand, why it is used inside the Replace Function
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810455 (1810455,)


In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)
new_arr, idx 

(array([[99.  ,  8.79, -9.66, ..., 99.  , 99.  , 99.  ],
        [ 4.08, -0.29,  6.36, ...,  0.34, -4.32,  1.07],
        [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
        ...,
        [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
        [99.  , 99.  , 99.  , ..., 99.  , 99.  , 99.  ],
        [ 2.43,  2.67, -3.98, ..., 99.  , 99.  , 99.  ]]),
 (array([14015, 22612, 16609, ..., 13667, 20911, 15500]),
  array([74, 64,  2, ..., 10,  6, 19])))

In [0]:
print(idx)
print(idx[0][1], idx[1][1])

(array([14015, 22612, 16609, ..., 13667, 20911, 15500]), array([74, 64,  2, ..., 10,  6, 19]))
22612 64


In [0]:
n_latent_factors = 50

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001, weight_decay=0.1):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * (latent_item_features[item_id])
    latent_item_features[item_id] -= alpha * err * (user_pref_values)
    return err
    

def sgd(iterations=100):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                # if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        # if (iteration % 100) == 0:
        #     print('MSE ', iteration, ': ', mse)
        print('MSE ', iteration, ': ', mse)


In [0]:
sgd()

MSE  0 :  28.499602560718575
MSE  1 :  25.03291368086652
MSE  2 :  24.94713944856626
MSE  3 :  24.87804649505107
MSE  4 :  24.817132151328895
MSE  5 :  24.757047271881433
MSE  6 :  24.690283471420283
MSE  7 :  24.60795927523813
MSE  8 :  24.498582194447685
MSE  9 :  24.346847029676212
MSE  10 :  24.132864745234592
MSE  11 :  23.832798269196395
MSE  12 :  23.422606601424253
MSE  13 :  22.886698169555878
MSE  14 :  22.23098502749563
MSE  15 :  21.493875878368044
MSE  16 :  20.743394927992995
MSE  17 :  20.054608660333262
MSE  18 :  19.480119862027028
MSE  19 :  19.035709666545117
MSE  20 :  18.707883855331676
MSE  21 :  18.470733484778858
MSE  22 :  18.298623738246388
MSE  23 :  18.17131898438214
MSE  24 :  18.074369287152212
MSE  25 :  17.997853710303026
MSE  26 :  17.93501021899553
MSE  27 :  17.88118892606616
MSE  28 :  17.83313828463562
MSE  29 :  17.788536214235716
MSE  30 :  17.74568450830639
MSE  31 :  17.703309230714478
MSE  32 :  17.660429957903844
MSE  33 :  17.61627422129409
M

In [0]:
def validation_check():
  error = []
  prediction = latent_user_preferences.dot(latent_item_features.T)
  for i in range(len(idx[0])):
    predicted_rating = prediction[idx[0][i]][idx[1][i]]
    actual_rating = arr[idx[0][i]][idx[1][i]]
    err = predicted_rating - actual_rating
    error.append(err)
  mse = (np.array(error) ** 2).mean()
  print('Validation MSE: ', mse)


In [0]:

validation_check()

Validation MSE:  17.40694818348049


In [0]:
### Creating new validation arrays from original data randomly again
for i in range(10):
  new_arr, idx = replace(arr, 0.1)
  validation_check()

Validation MSE:  13.164094672889279
Validation MSE:  13.178026007754523
Validation MSE:  13.143826322899601
Validation MSE:  13.200318830280587
Validation MSE:  13.133465276417263
Validation MSE:  13.131730838361424
Validation MSE:  13.177710839217866
Validation MSE:  13.186039507489362
Validation MSE:  13.156025456110102
Validation MSE:  13.221825704730675


In [0]:
prediction = latent_user_preferences.dot(latent_item_features.T)

In [0]:
### Prediction values on entire Test Dataset

values = [zip(df.values[i], prediction[i]) for i in range(prediction.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -0.28740648663788665)","(8.79, 0.009445338931876227)","(-9.66, -8.59853889761772)","(-8.16, -9.884126196795496)","(-7.52, -9.444341705357301)","(-8.5, -2.6600499571587704)","(-9.85, -8.395222682876135)","(4.17, -2.5593709647138323)","(-8.98, -7.808209923535566)","(-4.76, -2.7283741294799055)","(-8.5, -3.081254288750779)","(-6.75, -0.4228891578608217)","(-7.18, -5.223885817268093)","(8.45, 1.9323204190901015)","(-7.18, -8.745811722611329)","(-7.52, -10.575185684076187)","(-7.43, -4.1811842559130445)","(-9.81, -5.318042700771052)","(-9.85, -8.247066805698598)","(-9.85, -10.085237346984247)","(-9.37, -0.33954749625024216)","(1.5, -1.8483820027868751)","(-4.37, -4.959095409235043)","(-9.81, -12.21311731262445)","(-8.5, -2.1743385383204976)","(1.12, 1.6442303882587548)","(7.82, 1.9000725854332747)","(2.86, -1.0245800780795105)","(9.13, 6.213421635913247)","(-7.43, -10.27094396724479)","(2.14, 1.2075988733094647)","(-4.08, 3.126288091850083)","(-9.08, -10.533054114110252)","(7.82, -0.9084375178494631)","(5.05, 3.438186954969728)","(4.95, 3.8505030002344647)","(-9.17, -7.033924924825838)","(-8.4, -1.9014462299765107)","(-8.4, -1.759961924641436)","(-8.4, -5.2997574320022265)",...,"(8.59, 1.1049332375465029)","(3.59, 3.095106958933404)","(-6.84, -5.263222293673999)","(-9.03, -9.755157044830028)","(2.82, 1.886376705804537)","(-1.36, 1.479376140443383)","(-9.08, -8.547007993843351)","(8.3, 2.2172290563428416)","(5.68, 4.7288177523653)","(-4.81, -4.127755841807165)","(99.0, -8.246228753986394)","(99.0, 0.31166855906487334)","(99.0, -7.215798567947634)","(99.0, -8.992255201030018)","(99.0, -6.572247741230524)","(99.0, -1.1295292815735656)","(99.0, -6.8403122720674725)","(-9.42, -6.3028169136162955)","(99.0, -7.4866200958051925)","(99.0, -4.708420381392995)","(99.0, -3.337531635465993)","(-7.72, -5.395435484246854)","(99.0, -1.5335689962035564)","(99.0, -5.666352410315536)","(99.0, -5.904030604285359)","(99.0, -4.854827381061481)","(99.0, -0.006816070155035841)","(99.0, -1.3886526767243748)","(99.0, 2.279449707733942)","(99.0, -7.899572482165331)","(2.82, -2.4543631242762323)","(99.0, -3.0831002509147143)","(99.0, 0.15961077249091876)","(99.0, -5.558021676107721)","(99.0, -7.0489033267658465)","(99.0, -1.9513127508974932)","(-5.63, -4.078645263107748)","(99.0, -2.090452221624449)","(99.0, -6.438743244516179)","(99.0, -3.1116849577270895)"
1,"(4.08, 6.948943070275182)","(-0.29, 1.0977187984502814)","(6.36, 7.647755816828737)","(4.37, 7.086016451825411)","(-2.38, 1.899888065188034)","(-9.66, 2.0409145487251674)","(-0.73, 0.4007352349022044)","(-5.34, 1.1693467835463762)","(8.88, 5.215616205313447)","(9.22, 6.999946965217845)","(6.75, 5.932185113310183)","(8.64, 1.392523320116595)","(4.42, 2.229687014028814)","(7.43, 1.4922391169565135)","(4.56, 2.126595339118432)","(-0.97, 1.7256567816893906)","(4.66, 1.6798581945335562)","(-0.68, -1.0751081932211017)","(3.3, 0.33747497601844756)","(-1.21, -2.544204155032083)","(0.87, 3.352505842343995)","(8.64, 5.770813266930272)","(8.35, 2.906672449191129)","(9.17, 8.965695093625298)","(0.05, 3.542617049883457)","(7.57, 1.8271916298391937)","(4.71, 1.7388412168587495)","(0.87, -1.8222161247757855)","(-0.39, -0.7746774406427854)","(6.99, 3.698345830780308)","(6.5, 2.404005988334623)","(-0.92, -1.221387581640754)","(7.14, 5.124437894723154)","(9.03, 6.115603567960874)","(-1.8, 0.7371999925955592)","(0.73, 1.3349994932772247)","(7.09, 3.1053966533485777)","(3.4, 3.781831875240397)","(-0.87, 3.0445063964362533)","(7.91, 4.375589019508437)",...,"(-6.7, 0.6222731158180873)","(-3.35, -2.4863218985858078)","(-9.03, -2.704492654326801)","(4.47, 4.918707857194208)","(4.08, 3.080358023838376)","(-3.83, -0.7885030400626056)","(8.74, 7.103718443714596)","(1.12, -2.79592465807

In [0]:
jokes_ratings = pd.DataFrame(prediction)
mean_idx = jokes_ratings.mean()
min_idx = mean_idx.idxmin()
max_idx = mean_idx.idxmax()
print('Worst Joke ID predicted: ', min_idx, '; Average Rating: ', mean_idx[min_idx])
print('Best Joke ID predicted: ', max_idx, '; Average Rating: ', mean_idx[max_idx])

Worst Joke ID predicted:  57 ; Average Rating:  -3.9873021561228197
Best Joke ID predicted:  49 ; Average Rating:  3.5181498782131433
